**PROCESS A TECHNICAL BUS RFP INTO A COMPLIANCE MATRIX**

**File Name**: 190905_RFP_Processor.ipynb

**Inputs:** 
- Excel document with these tabs: meta, toc and technical
- Compliance Matrix Bank

**Outputs:** Compliance Matrix with compliance indications and PAE Suggestions

# ESTABLISMENT

**Imports**

In [389]:
# EXTERNAL FILES
import glob
# import pickle
import os
import requests # was use to handle proxies

import sys
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from D3AI_A7_utils import AIA7_190000_Module as utils

# DEFAULT DATA SCICENCE
import pandas as pd
import numpy as np
import random
import re
import time

# PLOTING
# Matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sn

import tensorflow as tf
import keras as keras
import keras.preprocessing.text as Tokenizer
import json
from gensim.models import Word2Vec
from typing import List
from sklearn.decomposition import PCA
import math
import nltk

import gensim
import collections
import smart_open
import collections
# from nltk.tokenize import sent_tokenize
# from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import warnings

# import tabula

In [390]:
def timex(startTm):
    print(f'Duration: {time.time() - startTm}')

## Global Parameters

In [391]:
timeStartPipe = time.time()

# COMMON:
maxlen = 100 # Max number of words in description
embedding_size = 300


# DEVELOPMENT:
# Transactional / Parser
# rfp_path = '../../nb_bid_data/190909_bid_pipe/input/'
# output_path = '../../nb_bid_data/190909_bid_pipe/output/'
rfp_path = ''
output_path = ''
# Keras Compliance Classifier
keras_tokenizer_path = '../../nb_bid_data/190909_bid_pipe/190927V3_keras_tokenizer.txt'
keras_classification_model_path = '../../nb_bid_data/190909_bid_pipe/190927V3_simple_nlp_keras_model.h5'
# Smooth Inverse Frequency
word2vec_path ='../../nb_bid_data/190909_similarity/190927_gensim_word2vec_embedding_model.bin'
bank_path = '../../nb_bid_data/190909_bid_pipe/190903_all_4_bid_compilations_dl.xlsx'


# # EXECUTABLE:
# # Transactional / Parser
# rfp_path = ''
# output_path = ''
# # Keras Compliance Classifier
# keras_tokenizer_path = '190927V3_keras_tokenizer.txt'
# keras_classification_model_path = '190927V3_simple_nlp_keras_model.h5'
# # Smooth Inverse Frequency
# word2vec_path ='190927_gensim_word2vec_embedding_model.bin'
# bank_path = '190903_all_4_bid_compilations_dl.xlsx'

## Input Data

# FORMULAS

## Transactional Formulas

**Input Formulas**

### format_headings()

In [392]:
def format_headings(ls):
    ls = [x.lower() for x in ls]
    ls = [x.strip() for x in ls]
    ls = [re.sub(" ", "_", x) for x in ls]
    return ls

### create_standard_df()

In [393]:
# Maid df
def create_standard_df():
    '''
    Creating an empty df to be use as the standard df to be pass in the pipes
    '''
    col_names = ['CID','HEADING','LIST','SPEC','TABLE COL 1','TABLE COL 2','TABLE COL 3','TABLE COL 4','TABLE COL 5','TABLE COL 6','TABLE COL 7',
                 'TABLE COL 8','TABLE COL 9','BLANK','BLACKLIST','BRAND','SECTION','SUB-SECTION','PAGE',
                 
                 'SALES ENG GROUP','AI SE COMPLIANCE','ACTION','TAG','NUMBER','MODEL','AI MODEL','AI VARIANT','PAE','SE COMMENT','SE FEEDBACK TO IT',
                 'AI PAE 1','AI PAE 1 SPEC','AI PAE 1 SOURCE','AI PAE 2','AI PAE 2 SPEC','AI PAE 2 SOURCE','AI PAE 3','AI PAE 3 SPEC','AI PAE 3 SOURCE',
                 'RATIONAL','PD REQUEST','PD COMMENT','AGENCY RESPONSE','AGENCY COMMENT/ADDENDUM',
                 
                 'PLC GROUP','AI ENG COMPLIANCE','PLC STATUS','PLC COMMENT',
                 'ENG OWNER','ENG COMPLIANT','ENG DATE','PROGRAM','REVIEW SIGNATURE','MEANS OF COMPLIANCE','ARGUMENT','INTERNAL COMMENT','PLC FEEDBACK TO IT',
                 
                 'PIT GROUP','ORG AI SE COMPLIANCE','ORG AI BLACKLIST','ORG AI BRAND','ORG AI TAG','ORG AI SECTION','ORG AI SUB-SECTION',
                 'ORG AI PAE 1 ID','ORG AI PAE 1 SCORE','ORG AI PAE 2 ID','ORG AI PAE 2 SCORE','ORG AI PAE 3 ID','ORG AI PAE 3 SCORE',
                 'ORG AI PD MEANS OF COMPLIANCE','ORG AI PD ARGUMENT','CAT','SPEC_ONE_COL','WORD NBR','ORG COL NBR','RFP NUMBER',
                 'RFP NAME','CLIENT NUMBER','ID']
    ls = format_headings(col_names)
    df = pd.DataFrame(columns=ls)
    df.head(2)
    print("     Created main dataframe.")
    return df, ls

### get_rfp()

In [394]:
def get_rfp(rfp_path):
    # Load all Word documents in the repo
    file_names = [{'path':pathTx} for pathTx in glob.glob(f'{rfp_path}*.xlsx')]
    # Eliminate temp files
    file_names = [bid for bid in file_names 
                if not next(iter(re.findall(r'~', bid['path'], flags=re.IGNORECASE)), None)]
    print(f'     Number of documents found: {len(file_names)} document(s)')
    # Documents to analyse
    print(f'     Processing document documents: {file_names}')
    file_names[0]['path']

    # Loading each line of the source file into a list
    # Currently only processing the first file
    print('     Loading the RFP...')
    meta = pd.read_excel(file_names[0]['path'], sheet_name='meta', header=None)
    toc = pd.read_excel(file_names[0]['path'], sheet_name='toc', header=None)
    tech = pd.read_excel(file_names[0]['path'], sheet_name='technical', header=None)
    print('     ...RFP loaded')
    return meta, toc, tech

### load_ref_files()

In [395]:
# def load_ref_files(keras_classification_model_path):
#     conformity_model = keras.models.load_model(keras_classification_model_path)

#     return conformity_model

**Output Formulas**

### formating_out_cid

In [396]:
def formating_out_cid(df):
    se = df.cid.str.replace(r'^\s*$', "", regex=True)
    mask = se.str.len() < 1
    df.loc[mask, 'cid'] = np.nan
    mask2 = df.cat.str.contains(r'para|list|table')
    df.loc[mask2,'cid'] = df.loc[mask2,'cid'].fillna(method='ffill')
    df.cid = df.cid.fillna(value="")
    return df

In [397]:
# tt = formating_out_cid(df)

In [398]:
# tt

In [399]:
# tt.info()

In [400]:
# mask\ = df.cat.str.contains(r'', regex=True)

In [401]:
# mask

In [402]:

# df.cid.fillna(method='ffill')
# df.cid.ffill()

### formating_tc()

In [403]:
def formating_out_tc(main):
    main['spec_one_col'] = main.spec
    mask = main.cat == 'table'
#     mask = ":"
    split_df = main.loc[:, 'spec'].str.split(pat='<tc>',expand=True)
    main.loc[:,'spec_one_col'] = main.loc[:,'spec'].str.replace(r'<tc>'," ")
#     breakpoint()
    for col in split_df:
        if col == 0:
            main.loc[mask,'spec'] = split_df[col]
        else:
            main.loc[mask,f'table_col_{col}'] = split_df[col]

    
#     for col in split_df:
# #         breakpoint()
#         if (col - spec_first_col) < 0:
#             breakpoint()
#             main.loc[:,'cid'] = main.loc[:,'cid'] + split_df[col]
#         elif (col - spec_first_col) == 0:
# #             breakpoint()
#             main.loc[:,'spec'] = split_df[col]
#         else:
# #             breakpoint()
#             main.loc[:,f'table_col_{col-spec_first_col}'] = split_df[col]
    return main

In [404]:
# df.groupby('cat').count()

In [405]:
# df.info()

In [406]:
# tt = formating_tc(df, spec_first_col)

In [407]:
# tt.tail(30)

### formating_heading()

In [408]:
def formating_heading(main):
    '''
    Move the headings in on distinc column
    Forward fill the headings
    '''
#     breakpoint()
    mask = main.cat == 'heading'
    main.loc[mask, 'heading'] = main.loc[mask, 'spec']
    main.loc[mask, 'spec'] = np.nan
    main.heading = main.heading.ffill()
#     main['admin_group'] = 'eof'
    return main

### saving_df()

In [409]:
def saving_df(df, rejected, columns_names_ls, output_path, df_file_name_prefix=None):
    df['id'] = df.index.values
    # Only keeping the standard format
    df = df[columns_names_ls]
    
    full_path = f'{output_path}{bid_number}_{bid_name}_output_raw.xlsx'
    writer = pd.ExcelWriter(full_path, engine='xlsxwriter')
    try:
        df.to_excel(writer, na_rep="", header=False, sheet_name='main', index=None)
#         rejected_full_path = f'{output_path}{df_file_name_prefix}_{bid_number}_{bid_name}_rejected_output.xlsx'
        if rejected.shape[0] > 0:
            rejected.to_excel(writer, na_rep="", header=True, sheet_name='rejected')
        writer.save()
        print(f'Document successfuly created!')
    except:
        print(f'ERROR: Document was NOT saved! Is the file currenlty open?')

    return _

## Parsing Formulas
These parsing formulas are conceived to be ran consecutively as per the below order.

### parse_raw_row_column()

In [410]:
# def parse_raw_row_column(main, column_name, raw):
#     '''
#     INPUTS:
#         main = Conformity df
#         columns_name = Name of the targeted feature
#         scramble = df created from an Acrobat PDF converted to an excel spreadsheet that need to be parsed
    
#     OUTPUT: Main df
    
#     Take a df created from an Acrobat PDF converted to an excel spreadsheet.
#     Convert that df into a list that become the rows of the df
#     Identify each cell beyond the first column as a table cell
#     '''
#     print("     Identifying lines and paragraphs...")

#     # Replace unrocognized caracters
#     raw = raw.replace({'\\uf0a7(?=\s)':'-'}, regex=True)
    
#     spec = []
#     cat = []
#     for index, row in raw.iterrows():
#         row_list = []
#         # Grouping all columns of the current row in one list
#         for col in row:
#             # testing of the value of the column is not np.nan
#             if not (col != col):
#                 row_list.append(col)
#         # if True, then it is a row of a table
#         if len(row_list) > 1:
#             text = "<tc>".join(str(x).strip() for x in row_list)
#             spec.append(text)
#         # if True then it is not a row of a table
#         # and it should be splited for in seperate spec is their is '\n'
#         elif len(row_list) == 1:
#             row_list = str(row_list[0]).split('\n')
# #             print(row_list)
#             spec = spec + row_list
#     main[column_name] = pd.Series(spec).astype(str)    
#     print(f'          ...Found {len(spec)} paragraphs') 
#     return main

In [411]:
def parse_raw_row_column(main, column_name, raw, spec_first_col):
    '''
    INPUTS:
        main = Conformity df
        columns_name = Name of the targeted feature
        scramble = df created from an Acrobat PDF converted to an excel spreadsheet that need to be parsed
    
    OUTPUT: Main df
    
    Take a df created from an Acrobat PDF converted to an excel spreadsheet.
    Convert that df into a list that become the rows of the df
    Identify each cell at spec_first_col position and beyond as a table cell
    '''
    print("     Identifying lines and paragraphs...")

    # Replace unrocognized caracters
    raw = raw.replace({'\\uf0a7(?=\s)':'-'}, regex=True)
    cid_ls = []
    spec_ls = []
    for index, row in raw.iterrows():
        row_list = []
        
        # Grouping all columns of the current row in one list
        for col in row:
            # testing of the value of the column is not np.nan
            if not (col != col):
                row_list.append(col)
                
        # if True, then it is a row of a table
        if len(row_list) > 1:
            cid = " ".join(str(x).strip() for x in row_list[:spec_first_col])
            text = "<tc>".join(str(x).strip() for x in row_list[spec_first_col:])
            cid_ls.append(cid)
            spec_ls.append(text)

        # if True then it is not a row of a table
        # and it should be splited for in seperate spec if their is '\n'
        elif len(row_list) == 1:            
            row_list = str(row_list[0]).split('\n')
            cid_ls = cid_ls + [" " for x in row_list]
            spec_ls = spec_ls + row_list
    
    main[column_name] = pd.Series(spec_ls).astype(str)    
    main['cid'] = pd.Series(cid_ls).astype(str)
    
    print(f'          ...Found {len(spec_ls)} paragraphs') 
    return main

In [412]:
# tt = parse_raw_row_column(df,'spec', tech, spec_first_col)

In [413]:
# tt.loc[100:150,'cid':'spec']

### parse_string()

In [414]:
def parse_string(main, column_name):
    print('     Parsing text...')
    main[column_name] = main[column_name].str.replace(u'\xa0', u' ')
    main[column_name] = main[column_name].str.replace(u"\'", u"'")
    main[column_name] = main[column_name].str.replace(r'\s{2,}', u" ")
    main[column_name] = main[column_name].str.strip()
    print(f'          ...Parsed: {main[column_name].size} entries')
    return main

### parse_toc()

In [415]:
def parse_toc(main, toc):
    '''
    toc: This is the df with the table of content
    Identify the headings in the main document from the toc df
    '''
    print('     Identifying the headings...')
    temp = pd.DataFrame(columns=['toc','text','page'])
    # The heading mayby in several columns
    temp = parse_raw_row_column(temp, 'toc', toc, 0)
    temp = parse_string(temp, 'toc')
#     breakpoint()
    # replacing the "...." of the spaces between the heading and its page number
    temp['toc'] = temp['toc'].str.replace(r'\.{3,}'," <toc> ")
    temp['toc'] = temp['toc'].str.replace(r'\s{6,}'," <toc> ")

    mask = temp['toc'].str.contains('<toc>')
    temp.loc[~mask,'toc'] =  + temp.loc[~mask,'toc'] + '<toc>'

    # extracting the headings and the page numbers
    temp['text'] = temp['toc'].str.extract(r'(^.*)(?<=<toc>)')
    temp['page'] = temp['toc'].str.extract(r'(?=<toc>)(.+$)') 

    # removing the <toc> anotation added at line 11
    temp['text'] = temp['text'].str.replace(r'<toc>', "")
    temp['page'] = temp['page'].str.replace(r'<toc>', "")

    temp = parse_string(temp, 'text')
    
    temp = count_se_word(temp, 'toc', 'word_nbr')
    
    # also extracting heading text for cases where heading number was put into the CID fields
#     temp2 = temp
#     temp2.text = temp2.str.extract 
    
#     breakpoint()
    print(f'     TOC contains {temp.shape[0]} headings')
    # adding the information on the main df
    for index, row in temp.iterrows():
        mask2 = main.spec.str.startswith(str(row.text))
        # Heading canditate should be less than 20 words
#         breakpoint()
        if int(row['word_nbr']) < 20 and (mask2 == True).any():
            idx = main.loc[mask2,'word_nbr'].idxmin()
            main.loc[idx,'cat'] = 'heading'
            main.loc[idx,'page'] = row.page    
    headings_nbr = main[main['cat'] == 'heading'].shape[0]
    print(f'          ...Matched {headings_nbr} headings from the main document against the TOC')    
    return main


In [416]:
# _ = parse_toc(df, toc)

### parse_list()

In [417]:
def parse_list(main):
    print('     Identifiying lists item...')
    mask = main['spec'].str.match(r'^•(?=\s)|^(?=\s)|^-(?=\s)|^[A-Za-z][\.\)](?=\s)|^[0-9]{1,3}[\.\)](?=\s)')
    main.loc[mask,'cat'] = 'list'
    main.loc[mask,'list'] = main.loc[mask,'spec'].str.extract(r'(^•(?=\s)|^(?=\s)|^-(?=\s)|^[A-Za-z][\.\)](?=\s)|^[0-9]{1,3}[\.\)](?=\s))')[0]
    main.loc[mask,'list'] = main.loc[mask,'list'].str.replace(r'^',"-")
    main.list = main.list.str.strip()
    main.loc[mask,'spec'] = main.loc[mask,'spec'].str.replace(r'^•(?=\s)|^(?=\s)|^-(?=\s)|^[A-Za-z][\.\)](?=\s)|^[0-9]{1,3}[\.\)](?=\s)',"")
    main.spec = main.spec.str.strip()
    print(f'          ...Found {main[mask].shape[0]} list items')
    return main

### parse_table()

In [418]:
def parse_table(main, col_name, spec_first_col):
    ''' 
    Classifying the "cat" attribute as table for table.
    '''
    print('     Identifiying tables...')
#     mask = main[col_name].str.contains(r'<tc\d{1,2}>')
    main.org_col_nbr = main[col_name].str.count('<tc>')
#     breakpoint()
    mask = main.org_col_nbr > spec_first_col
    main.loc[mask,'cat'] = 'table'
    print(f'          ...Found {main[mask].shape[0]} table rows')
    return main            
    

### parse_page()

In [419]:
# # Sentence are cut (i.e. PDF page break...) merge back together
# def parse_page(main):
#     print(f'     Correcting pages breaks...')                    
#     maxIter = main.shape[0] -1
#     main['cat'] = main['cat'].fillna('para')
#     new_df = main[0:0]
#     # The for loops need consecutive index
#     main = main.reset_index(drop=True)
#     skip = False
#     i = 0
#     for index, row in main.iterrows():
#         # Checking current line
#         if not re.search(r'\.$|\?$|\!$|\$:', row.spec) and row['cat'] == 'para' and index < maxIter and skip == False:
#             # Checking next line                
#             if main.loc[index+1,'cat'] == 'para' and not re.search(r'^[A-Z]', main.loc[index+1,'spec']):
# #             and not re.search(r'^[A-Z,a-z]{1,3}.\d{1,3}.\d{1,3}\.?\s{1,3}\w*|^\d{1,3}.\d{1,3}\.?\s{1,3}\w*', main.loc[index+1,'spec']):
#                 # Merging current row and next row specs
#                 main.loc[index,'spec'] = str(main.loc[index,'spec']) +" "+str(main.loc[index+1,'spec'])
#                 skip = True
#                 main.loc[index,'dlt'] = False
#                 i += 1
#             # If skip is true, it was set true at the last operation
#             # Do noting with the spec and reset to skip to false
#             else:
#                 main.loc[index,'dlt'] = False
#         elif skip == True:
#             skip = False
#             main.loc[index,'dlt'] = True
#         else:
#             main.loc[index,'dlt'] = False
#     dlt = main[main['dlt'] == True]
#     # Deleting extrat columns
#     main = main.loc[main['dlt'] == False,:'id']
#     print(f'          ...Number of pages breaks corrected: {i}')
#     return main, dlt

In [420]:
# Sentence are cut (i.e. PDF page break...) merge back together
def parse_page(main):
    print(f'     Correcting pages breaks...')                    
    maxIter = main.shape[0]
    main['cat'] = main['cat'].fillna('para')
#     new_df = main[0:0]
    # The for loops need consecutive index
    main = main.reset_index(drop=True)
    main['dlt'] = False
    i = 0
    j = 0
    k = 0
    itr = False
    while True:
        if not re.search(r'\.$|\?$|\!$|\$:', main.loc[i,'spec']) and main.loc[i,'cat'] == 'para':
            if main.loc[i+1,'cat'] == 'para' and not re.search(r'^[A-Z]', main.loc[i+1,'spec']):
                main.loc[i,'spec'] = str(main.loc[i,'spec']) +" "+str(main.loc[i+1,'spec'])
                main.loc[i+1,'dlt'] = True
                itr = True
                k += 1
                
        # NLTK Sentence Tokenizer Merge
#         if not re.search(r'\.$|\?$|\!$|\$:', main.loc[i,'spec']) and main.loc[i,'cat'] == 'para':                            
#             if main.loc[i+1,'cat'] == 'para' and not re.search(r'^[A-Z]', main.loc[i+1,'spec']):
#                 main.loc[i,'spec'] = str(main.loc[i,'spec']) +" "+str(main.loc[i+1,'spec'])
#                 main.loc[i+1,'dlt'] = True
#                 itr = True
#                 k += 1
        j += 1
        i += 1
        if i >= maxIter:
#             print(j)
            if itr == False:
                break
            else:
                itr = False
            i = 0
        main = main.loc[main['dlt'] == False,:]
        main = main.reset_index(drop=True)
        maxIter = main.shape[0]
    print(f'          ...Number of pages breaks corrected: {k} in {j/i} loop(s)')
    return main

In [421]:
# tt = parse_page(df)

In [422]:
# gg, dlt = tt


In [423]:
# gg.loc[18,'spec']

In [424]:
# gg.head(30)

### parse_heading()

In [425]:
def parse_heading(df):
    '''
    Look for matching heading patterns for heading not in the TOC
    '''
    print(f'     Looking for heading not captured by TOC...')
    mask = df.spec.str.contains(r'^\w{1,3}.\d{1,3}.*') & (df.word_nbr < 13) & df.cat.str.contains('para') 
#     mask = df.cat.str.contains('para')
    df.loc[mask,'cat'] = 'heading'
#     breakpoint()
    print(f'     ... found {df[mask].shape[0]} headings not captured by TOC.')
    return df

In [426]:
# df.groupby('cat').count()

In [427]:
# parse_heading(df)

### parse_paragraph()


In [428]:
def parse_paragraph(main):
    '''
    Parsing large paragraphs to paragraphs with less than 75 words
    when possible (require the paragraph to have more than 1 sentence to be split).
    '''
    print(f'     Current document has {main.shape[0]} rows. Now spliting long paragraphs...')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()
    #     main['cat'] = main['cat'].fillna('para')
        new_ls = main[['cid','list','spec','cat','page']].values.tolist()
        itr = True
        itr_counter = 0
        split_counter = 0
        re_split = re.compile(r'\.\s{1,3}(?=[A-Z])')
        while itr and itr_counter < 10:
            itr_counter += 1
            current_ls = new_ls
            new_ls = []
            itr = False
            for idx, ls in enumerate(current_ls):
                try:
                    words_nbr = count_word(ls[2])
                except:
                    breakpoint()
                # count number of sentences
                sentences = re_split.split(ls[2])
                sentences_nbr = len(sentences) if isinstance(sentences, list) else 0
                # large paragraphs are split in the middle
                if (ls[3] == 'para' or ls[3] == 'list') and words_nbr > 75 and sentences_nbr > 1:
                    split_counter += 1
                    middle = find_list_middle(sentences)
                    itr = True
                    # Split the spec into to spec in the middle.
                    new_ls.append([ls[0],ls[1], '. '.join(sentences[:middle])+'.', ls[3], ls[4]])
                    new_ls.append([ls[0],ls[1], '. '.join(sentences[middle:]), ls[3], ls[4]])
                else:
                    new_ls.append(ls)
            temp = pd.DataFrame(new_ls, columns=['cid','list','spec','cat','page'])
            main = main.iloc[0:0]
            main.cid = temp.cid
            main.list = temp.list
            main.spec = temp.spec
            main.cat = temp.cat
            main.page = temp.page
        print(f'          ...Splited {split_counter} paragraphs over {itr_counter} document cycles. Document has now {main.shape[0]} rows. ')
    return main

In [429]:
# tt = parse_paragraph(df)

In [430]:
# tt.head()

### parse_out_text()

In [431]:
def parse_out_text(df):
    '''
    Removing data that should not make it into the Compliance Matrix
    '''
    print('     Removing unwanted text (i.e. page numbers in the footers)...')

    # Match "Page XX of XX" and other variants of this
    # Remove spec with no text
    mask = (df.spec.str.contains(r'^Page\s{1,3}\d{1,4}\sof\s{1,3}\d{1,4}$|^Page\s{1,3}\d{1,4}$|^p.\s{1,3}\d{1,4}$', flags=re.IGNORECASE)) | (df.word_nbr == 0)
    if (mask == True).any():
        rejected = df[mask]
        df = df[~mask]
    else:
        rejected = df[0:0]
    
    print(f'          ...Rejected {rejected.shape[0]} entries.')
    return df, rejected

In [432]:
# parse_out_text(df)

## Features Formulas

### tokenize()

In [433]:
def tokenize(df):
    print("     Tokenize specs...")
#     breakpoint()
    for idx, row in df.iterrows():
        try:
            tt = sent_tokenize(row.spec)[0]
        except:
            breakpoint()
    df['spec_tokens'] = df.spec.apply(lambda x: sent_tokenize(str(x))[0])
    df['spec_tokens'] = df.spec_tokens.str.lower()
    df['spec_tokens'] = df.spec_tokens.str.replace(r'\.\!\:\?', "")
    # split into words
    df['spec_tokens'] = df.spec_tokens.apply(lambda x: word_tokenize(str(x)))
    # remove remaining tokens that are not alphabetic
    df['spec_tokens'] = df.spec_tokens.apply(lambda ls: [token for token in ls if len(token) > 1])
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    df.spec_tokens = df.spec_tokens.apply(lambda ls: [w for w in ls if not w in stop_words])
    print("    ...Filtered stop words.")
    # stemming of words
    porter = PorterStemmer()
    df['spec_tokens'] = df.spec_tokens.apply(lambda ls: [porter.stem(token) for token in ls])
    print("    ...Stemmed word.")
    return df

In [434]:
# df.info()

In [435]:
# df = tokenize(df)

### add_meta()

In [436]:
def add_meta(main, meta):
    print('     Collecting RFP information...')
#     breakpoint()
    spec_first_col = meta.loc[meta[0] == 'Spec First Column',1].iloc[0]
    number = meta.loc[meta[0] == 'RFP Number',1].iloc[0]
    name = meta.loc[meta[0] == 'RFP Name',1].iloc[0]
#     breakpoint()
    main['rfp_number'] = number
    main['rfp_name'] = name
#     df['eng_date'] = str(number)[:2]
    print('          ...Project is: ',number)
    print('          ...Project Name is: ',name)
    return main, number, name, spec_first_col

In [437]:
# df, bid_number, bid_name, spec_first_col = add_meta(df, meta)

### add_id()

In [438]:
def add_id(main):
    print('     Creating specs ids...')
    main.reset_index(drop=True)
#     main['id'] = main.rfp_number + main.index/10000
#     main = main.set_index('id')

    main.id = main.index
    print(f'          ...Created {main.shape[0]} ids.')
    return main

## Qualifying Formulas

### count_se_word()

In [439]:
def count_se_word(main, col_name, word_count_col_name):
#     breakpoint()
    main[word_count_col_name] = main[col_name].str.split().apply(lambda x: len(x) if isinstance(x, list) else 0)
    return main

### count_word()

In [440]:
def count_word(st):
    words = st.split()
    return len(words) if isinstance(words, list) else 0

### find_list_middle()

In [441]:
def find_list_middle(input_list):
    middle = float(len(input_list))/2
    if middle % 2 != 0:
        return int(middle + .5)
    else:
        return int(middle)

### fxn()

In [442]:
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

In [443]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

## S Eng Classification

### tokenize_df()

In [444]:
def tokenize_df(df, feature_name):
    feature_tokens = feature_name+'_tokens'
    df[feature_tokens] = df[feature_name].apply(lambda x: sent_tokenize(str(x)))

    df[feature_tokens] = df[feature_tokens].astype(str).str.lower()

    df[feature_tokens] = df[feature_tokens].astype(str).str.replace(r'\.\!\:\?', "")

    # split into words
    df[feature_tokens] = df[feature_tokens].apply(lambda x: word_tokenize(str(x)))

    # remove remaining tokens that are not alphabetic
    df[feature_tokens] = df[feature_tokens].apply(lambda ls: [token for token in ls if len(token) > 1])

    # filter out stop words
    stop_words = set(stopwords.words('english'))
    df[feature_tokens] = df[feature_tokens].apply(lambda ls: [w for w in ls if not w in stop_words])

    # stemming of words
    porter = PorterStemmer()
    df[feature_tokens] = df[feature_tokens].apply(lambda ls: [porter.stem(token) for token in ls])

    return df


### wrangle_for_keras()

In [445]:
def wrangle_for_keras(df, feature, bool_input, maxlen):
    text = df[feature+'_idx'].values

    # pad to max lenght of max_seq_length
    tokens_padded = tf.keras.preprocessing.sequence.pad_sequences(text, maxlen=maxlen, padding='post')

    # This is the test data to use with the bool_input layer
#     bl = pd.get_dummies(df['pd_date']).as_matrix()
    bl = pd.DataFrame(0, index=np.arange(df.shape[0]), columns=bool_input)
#     bl = pd.concat([df, bd], axis=0)

    print('Padded docs: ', tokens_padded[0:1])
    print('padded docs shape: ', tokens_padded.shape)
    return tokens_padded, bl

### predict_compliance()

In [446]:
def predict_compliance(df, tokens, boolean, k_model):
    print('     Starting to classify the conformity')
    compliance_model = keras.models.load_model(keras_classification_model_path)
    compliance = compliance_model.predict([tokens,boolean], verbose=1)
    # The model has 2 outputs in a list. The first one is only to train the lstm layer
    # So it is discarded with the [1]
    # 
    df['ai_se_comp'] = df['ai_se_comp'] = compliance[1][:,0]
    # Only keeping the probability of compliance
    print(f'          ...predicted {df.shape[0]} specs')
    return df

### run_classifier()

In [447]:
def run_classifier(df, keras_tokenizer_path, keras_classification_model_path):
    df = df.dropna(subset=['spec'])
    with open(keras_tokenizer_path, 'r') as infile:
        json_tx = json.load(infile)
    t = keras.preprocessing.text.tokenizer_from_json(json_tx)
    k_model = tf.keras.models.load_model(keras_classification_model_path)
    # Only 'para' cat row are classified
    mask = df.cat.str.contains(r'para|list') & (df.word_nbr > 15)
    docs = t.texts_to_sequences(df.loc[mask, 'spec_tokens'].tolist())
#     breakpoint()    
    docs = keras.preprocessing.sequence.pad_sequences(docs, maxlen=100, padding='post')
    pred = k_model.predict(docs, verbose=True)
    df.loc[mask, 'ai_se_compliance'] = pred
    return df

In [448]:
# help(df.cat.str.contains)

## SIF Formulas
Using Gensim Word2Vec to create the embeddings

Inspired from: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

### tokenize_with_nltk()

In [449]:
# def tokenize_with_nltk(df, feature):
#     # TO-DO Should I build my own tokenizer?
#     # TO-DO add lemmentization
#     df.loc[:,:] = df.dropna(subset=[feature])
#     tokenizer = RegexpTokenizer(r'\w+')
#     df[feature+'_tokens'] = df[feature].apply(lambda x: tokenizer.tokenize(str(x)))
#     return df

### Class Word

In [450]:
# an embedding word with associated vector
class Word:
    def __init__(self, text, vector):
        self.text = text
        self.vector = vector

    def __str__(self):
        return self.text + ' : ' + str(self.vector)

    def __repr__(self):
        return self.__str__()

### Class Sentence

In [451]:
# a sentence, a list of words
class Sentence:
    def __init__(self, word_list):
        self.word_list = word_list

    # return the length of a sentence
    def len(self) -> int:
        return len(self.word_list)

    def __str__(self):
        word_str_list = [word.text for word in self.word_list]
        return ' '.join(word_str_list)

    def __repr__(self):
        return self.__str__()

### get_word_frequency()

In [452]:
# todo: get a proper word frequency for a word in a document set
# or perhaps just a typical frequency for a word from Google's n-grams
def get_word_frequency(word_text):
    return 0.0001  # set to a low occurring frequency - probably not unrealistic for most words, improves vector values

### sentence_to_vec()

In [453]:
# A SIMPLE BUT TOUGH TO BEAT BASELINE FOR SENTENCE EMBEDDINGS
# Sanjeev Arora, Yingyu Liang, Tengyu Ma
# Princeton University
# convert a list of sentence with word2vec items into a set of sentence vectors
def sentence_to_vec(sentence_list: List[Sentence], embedding_size: int, a: float=1e-3):
    sentence_set = []
    sentence_set_id = []    
    for dc in sentence_list:
        sentence = dc['vec']
        vs = np.zeros(embedding_size)  # add all word2vec values into one vector for the sentence
        sentence_length = sentence.len()
        for word in sentence.word_list:
            a_value = a / (a + get_word_frequency(word.text))  # smooth inverse frequency, SIF
            vs = np.add(vs, np.multiply(a_value, word.vector))  # vs += sif * word_vector

        vs = np.divide(vs, sentence_length)  # weighted average
        sentence_set.append(vs)  # add to our existing re-calculated set of sentences
        sentence_set_id.append(dc['id'])

    # calculate PCA of this sentence set
#     if len(sentence_set) > 0:
    pca = PCA()
#     breakpoint()
    pca.fit(np.array(sentence_set))
    u = pca.components_[0]  # the PCA vector
    u = np.multiply(u, np.transpose(u))  # u x uT

    # pad the vector?  (occurs if we have less sentences than embeddings_size)
    if len(u) < embedding_size:
        for i in range(embedding_size - len(u)):
            u = np.append(u, 0)  # add needed extension for multiplication below

    # resulting sentence vectors, vs = vs -u x uT x vs
    sentence_vecs = []
    for i, vs in enumerate(sentence_set):
        sub = np.multiply(u,vs)
        sentence_vecs.append({'id':sentence_set_id[i], 'vec':np.subtract(vs, sub)})
    return sentence_vecs

#     return _

### l2_dist()

In [454]:
# euclidean distance between two vectors
def l2_dist(v1, v2):
    sum = 0.0
    if len(v1) == len(v2):
        for i in range(len(v1)):
            delta = v1[i] - v2[i]
            sum += delta * delta
        return math.sqrt(sum)

### tokens_2_word_vectors()

In [455]:
def tokens_2_word_vectors(df, feature, word2vec):
    # convert the tokens to their vectors
    sentence_list = []
    for idx, row in df.iterrows():
        word_list = []
#         breakpoint()
        for word in row[feature]:
            if word in word2vec.wv.vocab:
                token = word2vec.wv[word]
                if len(token) > 1:  # ignore OOVs
                    word_list.append(Word(word, token))
                else:
                    print('ERROROORORORORORO')
        if len(word_list) > 0:  # did we find any words (not an empty set)
            sentence_list.append({'id':idx, 'vec': Sentence(word_list)})
    return sentence_list

### build_sentence_vector_dc()

In [456]:
def build_sentence_vector_dc(sentence_list, embedding_size):
    # apply single sentence word embedding
#     sentence_vector_lookup = dict()
    sentence_vectors = sentence_to_vec(sentence_list, embedding_size)  # all vectors converted together
#     if len(sentence_vectors) == len(sentence_list): # There is an error if not
#         for i in range(len(sentence_vectors)):
#             # map: text of the sentence -> vector
#             sentence_vector_lookup[sentence_list[i].__str__()] = sentence_vectors[i]
    return sentence_vectors

### generate_embeddings()

In [457]:
def generate_embeddings(df, embedding_size, word2vec_path, bank_path):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()

        # Getting doc vectors for the current RFP specs
        mask = df.cat.str.contains(r'para|list') & (df.word_nbr > 15)
        print(f"     Vectorizing {df[mask].shape[0]} bid specs...")
        word2vec = Word2Vec.load(word2vec_path)
        bid_sentence_list = tokens_2_word_vectors(df[mask], 'spec_tokens', word2vec)
        bid_sentence_vector_ls = build_sentence_vector_dc(bid_sentence_list, embedding_size)

        # Getting doc vectors for the specs in the bank of historical compliance matrix
        bank = pd.read_excel(bank_path)
        bank['spec_tokens'] = np.nan
        mask2 = bank.PAE.str.len() > 10
        print(f"     Vectorizing {bank[mask2].shape[0]} historical specs...")
        bank = tokenize_df(bank[mask2], "spec")
        bank_sentence_list = tokens_2_word_vectors(bank[mask2], 'spec_tokens', word2vec)
        bank_sentence_vector_ls = build_sentence_vector_dc(bank_sentence_list, embedding_size)
        print(f"Total of {bank[mask2].shape[0] * df[mask].shape[0]} vectorial combinations ")
    return bid_sentence_list, bid_sentence_vector_ls, bank_sentence_list, bank_sentence_vector_ls, bank

### generate_eucledians()

In [458]:
def generate_eucledians(df, bank, bid_sentence_vector_ls, bank_sentence_vector_ls):
    # Initial call to print 0% progress
#     print(f'Number of specs to evaluate: {len(bid_sentence_vector_ls)}')
    printProgressBar(0, len(bid_sentence_vector_ls), prefix = 'Progress:', suffix = 'Complete', length = 50)
    i = 0
    for bid_dc in bid_sentence_vector_ls:
    #     print()
        best_euclidean_dist = 6
        # initializing deque 
        best_match = collections.deque(maxlen=3)
        # Update Progress Bar
        printProgressBar(i + 1, len(bid_sentence_vector_ls), prefix = 'Progress:', suffix = 'Complete', length = 50)

        for bank_dc in bank_sentence_vector_ls:
            # If we found the best
            euclidean_dist = l2_dist(bid_dc['vec'], bank_dc['vec'])
            if  euclidean_dist < best_euclidean_dist:
                best_euclidean_dist = euclidean_dist
                best_match.appendleft((bank_dc['id'], euclidean_dist))
        i += 1
        if len(best_match) > 0:
            df.loc[bid_dc['id'], 'ai_pae_1'] = bank.loc[best_match[0][0],'PAE']
            df.loc[bid_dc['id'], 'ai_pae_1_spec'] = bank.loc[best_match[0][0],'spec']
            df.loc[bid_dc['id'], 'ai_pae_1_source'] = bank.loc[best_match[0][0],'Client']            
            df.loc[bid_dc['id'], 'org_ai_pae_1_id'] = best_match[0][0]
            df.loc[bid_dc['id'], 'org_ai_pae_1_score'] = best_match[0][1]
        if len(best_match) > 1:
            df.loc[bid_dc['id'], 'ai_pae_2'] = bank.loc[best_match[1][0],'PAE']
            df.loc[bid_dc['id'], 'ai_pae_2_spec'] = bank.loc[best_match[1][0],'spec']
            df.loc[bid_dc['id'], 'ai_pae_2_source'] = bank.loc[best_match[1][0],'Client']            
            df.loc[bid_dc['id'], 'org_ai_pae_2_id'] = best_match[1][0]
            df.loc[bid_dc['id'], 'org_ai_pae_2_score'] = best_match[1][1]
        if len(best_match) > 2:
            df.loc[bid_dc['id'], 'ai_pae_3'] = bank.loc[best_match[2][0],'PAE']
            df.loc[bid_dc['id'], 'ai_pae_3_spec'] = bank.loc[best_match[2][0],'spec']
            df.loc[bid_dc['id'], 'ai_pae_3_source'] = bank.loc[best_match[2][0],'Client']            
            df.loc[bid_dc['id'], 'org_ai_pae_3_id'] = best_match[2][0]
            df.loc[bid_dc['id'], 'org_ai_pae_3_score'] = best_match[2][1]
    return df

### remove_poor_similarities()

In [459]:
def remove_poor_similarities(df):
    # Small sentences should be filtered out even is similar score is good
    mask = (df.org_ai_pae_1_score < 0.75) & (df.word_nbr > 10)
    df.loc[~mask,'ai_pae_1'] = np.nan
    df.loc[~mask,'ai_pae_1_spec'] = np.nan
    df.loc[~mask,'ai_pae_1_source'] = np.nan

    mask = (df.org_ai_pae_2_score < 0.75) & (df.word_nbr > 10)
    df.loc[~mask,'ai_pae_2'] = np.nan
    df.loc[~mask,'ai_pae_2_spec'] = np.nan
    df.loc[~mask,'ai_pae_2_source'] = np.nan

    mask = (df.org_ai_pae_3_score < 0.75) & (df.word_nbr > 10)
    df.loc[~mask,'ai_pae_3'] = np.nan
    df.loc[~mask,'ai_pae_3_spec'] = np.nan
    df.loc[~mask,'ai_pae_3_source'] = np.nan
    return df

In [460]:
# tt =remove_poor_similarities(df)

In [461]:
# df.info()

# PIPE

In [462]:
# time.sleep(20)

# Transactional formulas
print('LOADING REQUIRED FILES:')
df, column_name_ls = create_standard_df() # empty df
meta, toc, tech = get_rfp(rfp_path)
df, bid_number, bid_name, spec_first_col = add_meta(df, meta)

# Parsing
print()
print('PARSING THE TECHNICAL REQUIREMENTS:')
df = parse_raw_row_column(df,'spec', tech, spec_first_col)
df = parse_string(df, 'spec')
df = count_se_word(df, 'spec', 'word_nbr') # Need word count value to properly select the heading in parst_toc()
df = parse_toc(df, toc)

df = parse_list(df)

df = parse_table(df, 'spec', spec_first_col)

# if spec_first_col == 0:
df = parse_page(df)
df = parse_heading(df)
df = parse_paragraph(df)
df = count_se_word(df, 'spec', 'word_nbr')
df, rejected = parse_out_text(df)

# Feature Management
df = add_id(df)
df, bid_number, bid_name, spec_first_col = add_meta(df, meta)

LOADING REQUIRED FILES:
     Created main dataframe.
     Number of documents found: 1 document(s)
     Processing document documents: [{'path': 'mississauga toc.xlsx'}]
     Loading the RFP...
     ...RFP loaded
          ...Project is:  10000
          ...Project Name is:  Mississauga

PARSING THE TECHNICAL REQUIREMENTS:
     Identifying lines and paragraphs...
          ...Found 1375 paragraphs
     Parsing text...
          ...Parsed: 1375 entries
     Identifying the headings...
     Identifying lines and paragraphs...
          ...Found 235 paragraphs
     Parsing text...
          ...Parsed: 235 entries
     Parsing text...
          ...Parsed: 235 entries
     TOC contains 235 headings
          ...Matched 228 headings from the main document against the TOC
     Identifiying lists item...
          ...Found 248 list items
     Identifiying tables...
          ...Found 27 table rows
     Correcting pages breaks...
          ...Number of pages breaks corrected: 169 in 3.028192371

In [463]:
print()
print('TEXT PREPROCESSING')
df = tokenize(df)

print()
print('CLASSIFYING COMPLIANCE WITH AI:')
df = run_classifier(df, keras_tokenizer_path, keras_classification_model_path)

print()
print('PAE SUGGESTIONS:')
bid_sentence_list, bid_sentence_vector_ls, bank_sentence_list, bank_sentence_vector_ls, bank = generate_embeddings(df, embedding_size, word2vec_path, bank_path)
df = generate_eucledians(df, bank, bid_sentence_vector_ls, bank_sentence_vector_ls)


TEXT PREPROCESSING
     Tokenize specs...
    ...Filtered stop words.
    ...Stemmed word.

CLASSIFYING COMPLIANCE WITH AI:
839/839 [==============================] - 1s 893us/sample

PAE SUGGESTIONS:
     Vectorizing 839 bid specs...
     Vectorizing 17452 historical specs...
Total of 14642228 vectorial combinations 
Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [464]:
df = remove_poor_similarities(df)

print()
print('FORMATING THE OUTPUTS:')
df = formating_out_tc(df)


FORMATING THE OUTPUTS:


In [465]:
df = formating_heading(df)
df = formating_out_cid(df)

df = count_se_word(df, 'spec_one_col', 'word_nbr')

_ = saving_df(df, rejected, column_name_ls, output_path, '190927')

Document successfuly created!


In [150]:
# df.loc[df.spec.str.contains('Replacement vehicles shall re-use equipment as specified in Section 9'),'spec']

# Summary Statistics

In [ ]:
df.head(30)

In [ ]:
df.info()

In [ ]:
df.groupby('cat').count()

In [ ]:
df.describe()

In [ ]:
df.loc[:,]

## Compliance Classifier

In [ ]:
print('Example of compliance predictions:')
for isx, row in df.sample(n=5).iterrows():
    print()
    print(f'Compliance: ', row.ai_se_compliance)
    print(f'Spec: ', row.spec)    

In [ ]:
print('Tokens examples: ',bank.spec_tokens[1120:1124])

# OLD

In [ ]:
df = run_classifier(df, keras_tokenizer_path, keras_classification_model_path)


#### doc2vec

In [ ]:
# bank.info()

In [ ]:
# bank.info()
desc_ls = bank.Description.tolist()

In [ ]:
def read_corpus(ls, tokens_only=False):
    for i, line in enumerate(desc_ls):
        if tokens_only:
            yield gensim.utils.simple_preprocess(line)
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [ ]:
doc2vec = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=15)

In [ ]:
train_corpus = list(read_corpus(descriptions))
# test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [ ]:
len(train_corpus)

In [ ]:
doc2vec.build_vocab(train_corpus)

In [ ]:
%time doc2vec.train(train_corpus, total_examples=doc2vec.corpus_count, epochs=doc2vec.epochs)

In [ ]:
type(train_corpus)

In [ ]:
most_sim = []
# second_doc = []
for i in range(50):
    doc =  random.choice(train_corpus)

#     first_doc.append(doc)
    inferred_vector = doc2vec.infer_vector(doc.words)
    sim = doc2vec.docvecs.most_similar([inferred_vector], topn=3)
#     breakpoint()
    most_sim.append({'spec':doc.tags[0], 'sim':sim})
    

In [ ]:
for x in most_sim:
#     x =  random.choice(most_sim)
#     breakpoint()
    print()
    print(f'MAIN  : {train_corpus[x["spec"]]}')
    print(f'MATCH : {x["sim"][0][1]}')
    print(f'SIM   : {train_corpus[x["sim"][0][0]]}')
    
# len(most_sim)/

In [ ]:
most_sim

In [ ]:
help(gensim.models.doc2vec.TaggedDocument)

In [ ]:
# second_ranks

In [ ]:
collections.Counter(ranks)

In [ ]:
# help(SentenceTransformer)

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
model = SentenceTransformer("C:\\Users\\a324448\\\.cache\\torch\\sentence_transformers\\public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-base-nli-mean-tokens.zip\\modules.zip")

#### word2Vec

In [ ]:
# # train model with Gensim Word2Vec
# model = Word2Vec(descriptions, size=300, min_count=1)
# # summarize the loaded model
# print(model)
word2vec = Word2Vec.load(word2vec_path)

In [ ]:
# summarize vocabulary
words = list(word2vec.wv.vocab)
print('word examples in the vocab: ',words[100:110])
# access vector for one word
print('First representations of the vector: ', word2vec['moisture'][0:10])

In [ ]:
word2vec.wv.most_similar('door')

In [ ]:
# help(word2vec.wv)